<u><b>The Code below is to be ran in colab and so the magic ! keys won't work in a normal notebook:</b></u>  

In [ ]:
# -------------------------------------------------------------
# 1. GPU Setup
# -------------------------------------------------------------
!nvidia-smi
#or fallback:
# import subprocess
# subprocess.run(["nvidia-smi"])

import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

# -------------------------------------------------------------
# 2. Pull Latest Repo
# -------------------------------------------------------------
!git clone "https://github.com/Zed-080/NeuralSpikeSorter-2009.git"
%cd NeuralSpikeSorter-2009

# or fallback 
# subprocess.run(["git", "clone", "https://github.com/Zed-080/NeuralSpikeSorter-2009.git"])

# -------------------------------------------------------------
# 3. Install Dependencies
# -------------------------------------------------------------
!pip install -r requirements.txt

# or fallback:
# subprocess.run(["pip", "install", "-r", "requirements.txt"])

# or manually:
# !pip install numpy scipy scikit-learn tensorflow matplotlib


# -------------------------------------------------------------
# 4. Run Scripts
# -------------------------------------------------------------
!python main_train.py
!python main_infer.py

# or fallback:
# subprocess.run(["python", "main_train.py"])
# subprocess.run(["python", "main_infer.py"])

# -------------------------------------------------------------
# 5. Save Results to Google Drive (unique filename each run)
# -------------------------------------------------------------
from google.colab import drive
import shutil, datetime

drive.mount('/content/drive')

def save_results(dataset="D1-D6", threshold=0.95, refractory=60, folder="Colab Notebooks"):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    threshold_str = str(threshold).replace('.', '_')
    zip_name = f"results_{dataset}_thr{threshold_str}_refr{refractory}_{timestamp}"
    output_path = f"/content/drive/MyDrive/{folder}/{zip_name}"
    shutil.make_archive(output_path, 'zip', 'outputs')
    print(f"Saved archive: {output_path}.zip")

# Example usage:
save_results(dataset="D2-D6", threshold=0.82, refractory=45)

# -------------------------------------------------------------
# 6. Commit Results to GitHub (outputs + metadata)
# -------------------------------------------------------------
import subprocess, datetime

def commit_results(threshold=0.95, refractory=60, script_run="main_train.py + main_infer.py"):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    gpu_info = subprocess.getoutput("nvidia-smi --query-gpu=name --format=csv,noheader")
    threshold_str = str(threshold).replace('.', '_')

    commit_message = (
        f"Run at {timestamp} | GPU: {gpu_info} | Scripts: {script_run} | "
        f"Threshold={threshold_str} | Refractory={refractory}"
    )

    # Add and commit outputs folder
    !git add outputs/*
    !git commit -m "{commit_message}"
    !git push origin main

# Example usage:
commit_results(threshold=0.82, refractory=45)
